In [40]:
import re
from pathlib import Path

import chromadb
from chromadb.utils.embedding_functions.openai_embedding_function import (
    OpenAIEmbeddingFunction,
)
from trafilatura import extract, fetch_url
from openai import OpenAI

BASE_URL = "http://localhost:8000/v1"
EMBEDDING_MODEL = "multilingual-e5-large-q4"
LLM_MODEL = "gemma-2-2b-it-q4"

cna_pattern = re.compile(
    r"# (?P<title>.*)\n\n(?P<content>.+)（.*）\d{7}", re.DOTALL | re.MULTILINE
)
# https://www.cna.com.tw/news/aspt/202408110037.aspx re between / and .aspx
cna_id_pattern = re.compile(r"/(\d+)\.aspx")

raw_data_dir = Path("raw_data")
with open(raw_data_dir.joinpath("urls.txt"), encoding="utf-8") as f:
    urls = [line.strip() for line in f.readlines()]

chromadb_client = chromadb.PersistentClient(path="./db")
openai_embedding_function = OpenAIEmbeddingFunction(
    api_base=BASE_URL,
    model_name=EMBEDDING_MODEL,
    api_key="sk-template",
)
openai_client = OpenAI(base_url=BASE_URL, api_key="sk-template")

In [28]:
# chromadb_client.delete_collection("cna-retrieval")
collection = chromadb_client.get_or_create_collection(
    name="cna-retrieval",
    metadata={"hnsw:space": "cosine"},
    embedding_function=openai_embedding_function,
)

In [31]:
collection.peek()

{'ids': ['202407163013-0',
  '202407163014-0',
  '202407163016-0',
  '202408110043-0'],
 'embeddings': [[0.03971979767084122,
   0.029011785984039307,
   -0.02393793687224388,
   -0.032592009752988815,
   0.04220892861485481,
   -0.009600387886166573,
   -0.006971932481974363,
   0.07188264280557632,
   0.05119522660970688,
   -0.005439356435090303,
   0.02986438013613224,
   0.036344002932310104,
   -0.012583478353917599,
   0.005577784031629562,
   -0.03363635390996933,
   0.018701057881116867,
   0.004189847502857447,
   0.014860177412629128,
   -0.011683141812682152,
   -0.027889201417565346,
   0.03142809867858887,
   -0.005358172580599785,
   -0.04444800317287445,
   -0.03729815408587456,
   -0.039202168583869934,
   -0.0421631820499897,
   -0.027244504541158676,
   -0.05231209844350815,
   -0.025121862068772316,
   -0.023283278569579124,
   -0.028143875300884247,
   0.0348038449883461,
   -0.020909782499074936,
   -0.04031361639499664,
   -0.023418374359607697,
   0.030154291540

In [12]:
def crawler(url: str) -> tuple[str, str]:
    downloaded = fetch_url(url)
    result = extract(
        downloaded,
        output_format="markdown",
        include_links=False,
        include_comments=False,
        include_tables=False,
    )

    match = cna_pattern.match(result)
    if match:
        title = match.group("title")
        content = match.group("content")
        return title, content
    raise ValueError(f"Failed to extract title and content from {url}")

In [ ]:
for url in urls:
    title, content = crawler(url)
    # 把 content 切割成 256 個字元一段加上 title
    content = [f"{title}\n"+content[i:i+256] for i in range(0, len(content), 256)]
    collection.add(
        documents=content,
        metadatas=[{"url": url, "title": title}]*len(content),
        ids=[cna_id_pattern.search(url).group(1)+f"-{i}" for i in range(len(content))],
    )

In [45]:
query_result = collection.query(
    query_texts=["誰是跆拳道唯一代表？"],
    n_results=1
)
query_result

{'ids': [['202407163016-0']],
 'distances': [[0.1449602847957535]],
 'metadatas': [[{'title': '羅嘉翎跆拳道唯一代表有責任 再闖奧運盼超越上屆\n\n2024/7/16 17:49（7/26 10:12 更新）\n\n（中央社台北16日電）羅嘉翎19歲首度登上奧運舞台就踢下銅牌，近一年傷勢影響讓她每次出賽都痛到落淚，仍先後在成都世大運、杭州亞運奪銀，身為巴黎奧運跆拳道唯一代表，她多了責任感，將力拚獎牌成色升級。\n\n羅嘉翎出生跆拳道世家，父親羅文祥是教練，哥哥羅堉誠也是跆拳道選手，從幼兒園起就在家裡的道館看著大家訓練、耳濡目染，也開啟跆拳道生涯，此次奧運爸爸羅文祥不僅備戰期就到高雄國家運動訓練中心陪練，也將到巴黎觀賽。\n\n羅嘉翎國小六年級就是全國冠軍，高中在世界青少年跆拳道錦標賽完成2連霸、為台灣第1人。\n\n羅嘉翎以19歲的年紀取得東京奧運跆拳道女子57公斤級參賽資格，不料初生之犢的她首度挑戰奧運就奪下銅牌，也成了台灣最耀眼的跆拳道新星。\n\n東京奧運的成功沒讓羅嘉翎停下腳步，她在去年世界跆拳道錦標賽、成都世大運、杭州亞運57公斤級都成功奪銀，狀況看似風光，卻造成身體傷勢惡化。\n\n羅嘉翎身上一直都有傷勢，年初確定遞補取得巴黎奧運資格後，也給了她治療養傷的時間，而大半年沒有出賽的她，將在出征法國巴黎前透過模擬賽來尋找比賽感覺。',
    'url': 'https://www.cna.com.tw/news/aspt/202407163016.aspx'}]],
 'embeddings': None,
 'documents': [['羅嘉翎跆拳道唯一代表有責任 再闖奧運盼超越上屆\n\n2024/7/16 17:49（7/26 10:12 更新）\n\n（中央社台北16日電）羅嘉翎19歲首度登上奧運舞台就踢下銅牌，近一年傷勢影響讓她每次出賽都痛到落淚，仍先後在成都世大運、杭州亞運奪銀，身為巴黎奧運跆拳道唯一代表，她多了責任感，將力拚獎牌成色升級。\n\n羅嘉翎出生跆拳道世家，父親羅文祥是教練，哥哥羅堉誠也是跆拳道選手，從幼兒園起就在家裡的道館看著大家訓練、耳濡目染，也開啟跆拳道生涯，此次奧運爸爸羅文祥不僅備戰期就到高雄國家運動訓練中心陪練，也將到巴黎觀賽

In [46]:
llm_result = openai_client.chat.completions.create(
    model=LLM_MODEL,
    messages=[
        {"role": "system", "content": "你是一個有用的助手，樂意使用正體中文回答問題。"},
        {"role": "user", "content": f"{query_result['documents'][0]}\n\n" + "跆拳道唯一代表？"},
    ]
)
llm_result.choices[0].message.content

'巴黎奧運跆拳道女子57公斤級的代表是 **羅嘉翎**。 \n'